In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

# EfficientNetB0

In [2]:
def mbconv(x, out_ch, expand_ratio, k, s, se_ratio=0.25):
    in_ch = int(x.shape[-1]) 

    # 1) Expansion
    x_exp = x
    if expand_ratio != 1:
        x_exp = layers.Conv2D(in_ch * expand_ratio, 1, padding="same", use_bias=False)(x_exp)
        x_exp = layers.BatchNormalization()(x_exp)
        x_exp = layers.Activation("swish")(x_exp)

    # 2) Depthwise (k×k, stride=s) 
    x_dw = layers.DepthwiseConv2D(kernel_size=k, strides=s, padding="same", use_bias=False)(x_exp)
    x_dw = layers.BatchNormalization()(x_dw)
    x_dw = layers.Activation("swish")(x_dw)

    # 3) Squeeze-and-Excitation
    mid = int(x_dw.shape[-1])  # 채널 수
    se = layers.GlobalAveragePooling2D(keepdims=True)(x_dw)
    se = layers.Conv2D(filters=max(1, int(mid * se_ratio)), kernel_size=1, activation="swish")(se)
    se = layers.Conv2D(filters=mid,                         kernel_size=1, activation="sigmoid")(se)
    x_se = layers.Multiply()([x_dw, se])

    # 4) Projection
    x_proj = layers.Conv2D(filters=out_ch, kernel_size=1, padding="same", use_bias=False)(x_se)
    x_proj = layers.BatchNormalization()(x_proj)

    # Residual (s=1 & channels match)
    if s == 1 and in_ch == out_ch:
        return layers.Add()([x, x_proj])
    return x_proj

def EfficientNetB0(input_shape=(224, 224, 3), num_classes=1000, dropout=0.2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = mbconv(x, 16, 1, 3, 1)

    x = mbconv(x, 24, 6, 3, 2)
    x = mbconv(x, 24, 6, 3, 1)

    x = mbconv(x, 40, 6, 5, 2)
    x = mbconv(x, 40, 6, 5, 1)

    x = mbconv(x, 80, 6, 3, 2)
    x = mbconv(x, 80, 6, 3, 1)
    x = mbconv(x, 80, 6, 3, 1)

    x = mbconv(x, 112, 6, 5, 1)
    x = mbconv(x, 112, 6, 5, 1)
    x = mbconv(x, 112, 6, 5, 1)

    x = mbconv(x, 192, 6, 5, 2)
    x = mbconv(x, 192, 6, 5, 1)
    x = mbconv(x, 192, 6, 5, 1)
    x = mbconv(x, 192, 6, 5, 1)

    x = mbconv(x, 320, 6, 3, 1)

    x = layers.Conv2D(1280, 1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)

In [3]:
model = EfficientNetB0()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 112, 112, 32)      │             128 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 112, 112, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d              │ (None, 112, 112, 32)      │             288 │ activation[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 112, 112, 32)      │             128 │ depthwise_conv2d[0][0]     │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 112, 112, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d      │ (None, 1, 1, 32)          │               0 │ activation_1[0][0]         │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 1, 1, 8)           │             264 │ global_average_pooling2d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 1, 1, 32)          │             288 │ conv2d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply (Multiply)           │ (None, 112, 112, 32)      │               0 │ activation_1[0][0],        │
│                               │                           │                 │ conv2d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 112, 112, 16)      │             512 │ multiply[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 112, 112, 16)      │              64 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 112, 112, 96)      │           1,536 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 8,465,864 (32.29 MB)

 Trainable params: 8,423,848 (32.13 MB)

 Non-trainable params: 42,016 (164.12 KB)

# EfficientNetB1

In [4]:
def EfficientNetB1(input_shape=(240, 240, 3), num_classes=1000, dropout=0.2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = mbconv(x, 16, 1, 3, 1)
    x = mbconv(x, 16, 1, 3, 1)

    x = mbconv(x, 24, 6, 3, 2)
    x = mbconv(x, 24, 6, 3, 1)
    x = mbconv(x, 24, 6, 3, 1)

    x = mbconv(x, 40, 6, 5, 2)
    x = mbconv(x, 40, 6, 5, 1)
    x = mbconv(x, 40, 6, 5, 1)

    x = mbconv(x, 80, 6, 3, 2)
    x = mbconv(x, 80, 6, 3, 1)
    x = mbconv(x, 80, 6, 3, 1)
    x = mbconv(x, 80, 6, 3, 1)

    x = mbconv(x, 112, 6, 5, 1)
    x = mbconv(x, 112, 6, 5, 1)
    x = mbconv(x, 112, 6, 5, 1)
    x = mbconv(x, 112, 6, 5, 1)

    x = mbconv(x, 192, 6, 5, 2)
    x = mbconv(x, 192, 6, 5, 1)
    x = mbconv(x, 192, 6, 5, 1)
    x = mbconv(x, 192, 6, 5, 1)
    x = mbconv(x, 192, 6, 5, 1)

    x = mbconv(x, 320, 6, 3, 1)
    x = mbconv(x, 320, 6, 3, 1)

    x = layers.Conv2D(1280, 1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)  # B1 보통 0.2
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)

In [5]:
model = EfficientNetB1()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 240, 240, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_65 (Conv2D)            │ (None, 120, 120, 32)      │             864 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_49        │ (None, 120, 120, 32)      │             128 │ conv2d_65[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_33 (Activation)    │ (None, 120, 120, 32)      │               0 │ batch_normalization_49[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_16           │ (None, 120, 120, 32)      │             288 │ activation_33[0][0]        │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_50        │ (None, 120, 120, 32)      │             128 │ depthwise_conv2d_16[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_34 (Activation)    │ (None, 120, 120, 32)      │               0 │ batch_normalization_50[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d_17   │ (None, 1, 1, 32)          │               0 │ activation_34[0][0]        │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_66 (Conv2D)            │ (None, 1, 1, 8)           │             264 │ global_average_pooling2d_… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_67 (Conv2D)            │ (None, 1, 1, 32)          │             288 │ conv2d_66[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply_16 (Multiply)        │ (None, 120, 120, 32)      │               0 │ activation_34[0][0],       │
│                               │                           │                 │ conv2d_67[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_68 (Conv2D)            │ (None, 120, 120, 16)      │             512 │ multiply_16[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_51        │ (None, 120, 120, 16)      │              64 │ conv2d_68[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_17           │ (None, 120, 120, 16)      │             144 │ batch_normalization_51[0]… │
│ (DepthwiseConv2D)             │                           │               

 Total params: 13,398,252 (51.11 MB)

 Trainable params: 13,336,204 (50.87 MB)

 Non-trainable params: 62,048 (242.38 KB)

In [6]:
def EfficientNetB2(input_shape=(260, 260, 3), num_classes=1000, dropout=0.3):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, strides=2, padding="same", use_bias=False)(inputs)  
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = mbconv(x, 16, 1, 3, 1)   
    x = mbconv(x, 16, 1, 3, 1)

    x = mbconv(x, 24, 6, 3, 2)
    x = mbconv(x, 24, 6, 3, 1)
    x = mbconv(x, 24, 6, 3, 1)

    x = mbconv(x, 48, 6, 5, 2)   
    x = mbconv(x, 48, 6, 5, 1)
    x = mbconv(x, 48, 6, 5, 1)

    x = mbconv(x, 88, 6, 3, 2)   
    x = mbconv(x, 88, 6, 3, 1)
    x = mbconv(x, 88, 6, 3, 1)
    x = mbconv(x, 88, 6, 3, 1)

    x = mbconv(x, 120, 6, 5, 1)  
    x = mbconv(x, 120, 6, 5, 1)
    x = mbconv(x, 120, 6, 5, 1)
    x = mbconv(x, 120, 6, 5, 1)

    x = mbconv(x, 208, 6, 5, 2)  
    x = mbconv(x, 208, 6, 5, 1)
    x = mbconv(x, 208, 6, 5, 1)
    x = mbconv(x, 208, 6, 5, 1)
    x = mbconv(x, 208, 6, 5, 1)

    x = mbconv(x, 352, 6, 3, 1)  
    x = mbconv(x, 352, 6, 3, 1)

    x = layers.Conv2D(1408, 1, padding="same", use_bias=False)(x)  # 1280 → 1408
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout)(x)   # B2 권장 0.3
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)

In [7]:
model = EfficientNetB2()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 260, 260, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_157 (Conv2D)           │ (None, 130, 130, 32)      │             864 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_118       │ (None, 130, 130, 32)      │             128 │ conv2d_157[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_79 (Activation)    │ (None, 130, 130, 32)      │               0 │ batch_normalization_118[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_39           │ (None, 130, 130, 32)      │             288 │ activation_79[0][0]        │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_119       │ (None, 130, 130, 32)      │             128 │ depthwise_conv2d_39[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_80 (Activation)    │ (None, 130, 130, 32)      │               0 │ batch_normalization_119[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d_41   │ (None, 1, 1, 32)          │               0 │ activation_80[0][0]        │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_158 (Conv2D)           │ (None, 1, 1, 8)           │             264 │ global_average_pooling2d_… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_159 (Conv2D)           │ (None, 1, 1, 32)          │             288 │ conv2d_158[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply_39 (Multiply)        │ (None, 130, 130, 32)      │               0 │ activation_80[0][0],       │
│                               │                           │                 │ conv2d_159[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_160 (Conv2D)           │ (None, 130, 130, 16)      │             512 │ multiply_39[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_120       │ (None, 130, 130, 16)      │              64 │ conv2d_160[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_40           │ (None, 130, 130, 16)      │             144 │ batch_normalization_120[0… │
│ (DepthwiseConv2D)             │                           │               

 Total params: 15,746,072 (60.07 MB)

 Trainable params: 15,678,504 (59.81 MB)

 Non-trainable params: 67,568 (263.94 KB)